In [1]:
import numpy as np
import pandas as pd
import json
from utils import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, GRU, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from sklearn import preprocessing

In [2]:
truth_df = pd.DataFrame(columns=['id','truthMedian','truthClass','truthMean'])
with open('data/truth.jsonl') as data:
    for labelobj in data:
        truth = json.loads(labelobj)
        truthlabel = {'id': truth['id'], 'truthMedian': truth['truthMedian'], 'truthClass': truth['truthClass'], 'truthMean': truth['truthMean']}
        truth_df = truth_df.append(truthlabel, ignore_index = True)
truth_df.head()  

,id,truthMedian,truthClass,truthMean
0,858464162594172928,1.000000,clickbait,1.000000
1,858462320779026433,0.000000,no-clickbait,0.133333
2,858460992073863168,0.333333,no-clickbait,0.400000
3,858459539296980995,0.333333,no-clickbait,0.266667
4,858455355948384257,0.000000,no-clickbait,0.000000


In [3]:
instances_df = pd.DataFrame(columns=['id','postText'])
with open('data/instances.jsonl') as data:
	for instanceobj in data:
		instance = json.loads(instanceobj)
		instancerow = {'id': instance['id'], 'postText': instance['postText']}
		instances_df = instances_df.append(instancerow, ignore_index=True)
instances_df.head()

,id,postText
0,858462320779026433,[UK’s response to modern slavery leaving victi...
1,858421020331560960,[this is good]
2,858368123753435136,"[The ""forgotten"" Trump roast: Relive his bruta..."
3,858323428260139008,[Meet the happiest #dog in the world!]
4,858283602626347008,[Tokyo's subway is shut down amid fears over a...


In [4]:
dataset = instances_df.join(truth_df.set_index('id'), on='id')
dataset = dataset.drop(labels='id',axis=1)
for i in range(len(dataset)):
    dataset['postText'].values[i] = dataset['postText'].values[i][0]
dataset['postText'].dropna(inplace=True)
dataset.head()

,postText,truthMedian,truthClass,truthMean
0,UK’s response to modern slavery leaving victim...,0.000000,no-clickbait,0.133333
1,this is good,1.000000,clickbait,1.000000
2,"The ""forgotten"" Trump roast: Relive his brutal...",0.333333,no-clickbait,0.466667
3,Meet the happiest #dog in the world!,1.000000,clickbait,0.933333
4,Tokyo's subway is shut down amid fears over an...,0.000000,no-clickbait,0.000000


In [5]:
# Convert the 'no-clickbait' or 'clickbait' to binary indicator
dataset['truthClass'] = dataset['truthClass'].apply(classToBinary)
# Convert floating number in 'truthMedian' column to integer
dataset['truthMedian'] = dataset['truthMedian'].apply(medianToInteger)
# Remove all punctuations and clear the text
dataset['postText'] = dataset['postText'].apply(cleanText)
dataset.head()

,postText,truthMedian,truthClass,truthMean
0,uks response to modern slavery leaving victims...,0,0,0.133333
1,this is good,3,1,1.000000
2,the forgotten trump roast relive his brutal 20...,1,0,0.466667
3,meet the happiest dog in the world,3,1,0.933333
4,tokyos subway is shut down amid fears over an ...,0,0,0.000000


In [6]:
# import the glove word embedding file
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B/glove.6B.100d.txt')

In [7]:
# length of titles with longest words
maxLen = maxLengthInPostText(dataset)
maxLen

25

In [8]:
# split the dataset to training and testing set
train, test = train_test_split(dataset, test_size=0.2)
X_train, Y_train, Y_train_mean = np.array(train["postText"].tolist()), np.array(train["truthMedian"].tolist()), np.array(train["truthMean"].tolist())
X_test, Y_test, Y_test_mean = np.array(test["postText"].tolist()), np.array(test["truthClass"].tolist()), np.array(test["truthMean"].tolist())
print(Y_train.shape)
print(Y_test.shape)

(15630,)
(3908,)


In [9]:
Indices = sentences_to_indices(X_train,word_to_index, maxLen)
print("X_Train_indices =\n", Indices.shape)

X_Train_indices =
 (15630, 25)


In [10]:
def ClickBait_BiGRU(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(input_shape, dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    # Propagate sentence_indices through your embedding layer
    embeddings = embedding_layer(sentence_indices)   
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    X = Bidirectional(GRU(128, return_sequences=True))(embeddings)
    # dropout
    X = Dropout(0.5)(X)
    X = Bidirectional(GRU(128, return_sequences=True))(X)
    # dropout
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output should be a single hidden state, not a batch of sequences.
    X = Bidirectional(GRU(128, return_sequences=False))(X)
    # dropout
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with 2 units
    X = Dense(4)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)  
    # Create Model instance which converts sentence_indices into X.
    model = Model(sentence_indices, X) 
    return model

In [11]:
BiGRU_model = ClickBait_BiGRU((maxLen,), word_to_vec_map, word_to_index)
BiGRU_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 25, 100)           40000100  
_________________________________________________________________
bidirectional (Bidirectional (None, 25, 256)           176640    
_________________________________________________________________
dropout (Dropout)            (None, 25, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 25, 256)           296448    
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               296448

In [12]:
BiGRU_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 4)
X_train_indices.shape

(15630, 25)

In [14]:
BiGRU_model.fit(X_train_indices, Y_train_oh, epochs = 10, batch_size = 32, shuffle=True)

Epoch 1/10
489/489 [==============================] - 18s 28ms/step - loss: 1.1004 - accuracy: 0.5114
Epoch 2/10
489/489 [==============================] - 15s 31ms/step - loss: 0.9563 - accuracy: 0.5750
Epoch 3/10
489/489 [==============================] - 16s 32ms/step - loss: 0.9282 - accuracy: 0.5818
Epoch 4/10
489/489 [==============================] - 16s 33ms/step - loss: 0.9184 - accuracy: 0.5912
Epoch 5/10
489/489 [==============================] - 16s 34ms/step - loss: 0.8768 - accuracy: 0.6046
Epoch 6/10
489/489 [==============================] - 16s 34ms/step - loss: 0.8602 - accuracy: 0.6093
Epoch 7/10
489/489 [==============================] - 16s 34ms/step - loss: 0.8301 - accuracy: 0.6329
Epoch 8/10
489/489 [==============================] - 17s 34ms/step - loss: 0.7954 - accuracy: 0.6454
Epoch 9/10
489/489 [==============================] - 17s 34ms/step - loss: 0.7340 - accuracy: 0.6755
Epoch 10/10
489/489 [==============================] - 17s 34ms/step - loss: 0.679

In [15]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,roc_auc_score, mean_squared_error, classification_report
y_train_pred_oh = BiGRU_model.predict(X_train_indices)
y_train_pred_binary = onehot_to_binary(y_train_pred_oh)
Y_train_binary = onehot_to_binary(Y_train_oh)

print("GRU Training Accuracy")
print('Accuracy %s' % accuracy_score(Y_train_binary, y_train_pred_binary))
print('Precision %s' % precision_score(Y_train_binary, y_train_pred_binary))
print('Recall %s' % recall_score(Y_train_binary, y_train_pred_binary))
print('F1 score: %s' % f1_score(Y_train_binary, y_train_pred_binary))
print('MSE %s' % mean_squared_error(Y_train_mean, y_train_pred_binary))

GRU Training Accuracy
Accuracy 0.927575175943698
Precision 0.8563829787234043
Recall 0.844700944386149
F1 score: 0.8505018489170627
MSE 0.08996374493220748


In [16]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
y_pred_onehot = BiGRU_model.predict(X_test_indices)
y_pred_binary = onehot_to_binary(y_pred_onehot)

print("GRU Testing Accuracy")
print('Accuracy %s' % accuracy_score(Y_test, y_pred_binary))
print('Precision %s' % precision_score(Y_test, y_pred_binary))
print('Recall %s' % recall_score(Y_test, y_pred_binary))
print('F1 score: %s' % f1_score(Y_test, y_pred_binary))
print('MSE %s' % mean_squared_error(Y_test_mean, y_pred_binary))

GRU Testing Accuracy
Accuracy 0.8346980552712385
Precision 0.6677740863787376
Recall 0.6354056902002108
F1 score: 0.6511879049676026
MSE 0.12188900261178279


In [17]:
print('Minimum MSE %s' % mean_squared_error(Y_test_mean, Y_test))

Minimum MSE 0.07733083134137338


In [18]:
print("GRU Testing report")
print(classification_report(Y_test, y_pred_binary))

GRU Testing report
              precision    recall  f1-score   support

           0       0.88      0.90      0.89      2959
           1       0.67      0.64      0.65       949

    accuracy                           0.83      3908
   macro avg       0.78      0.77      0.77      3908
weighted avg       0.83      0.83      0.83      3908



In [19]:
# Save our model
BiGRU_model.save("BiGRU_model.h5")

In [20]:
# Error Analysis
for i in range(10):
    if Y_test[i] - y_pred_binary[i] != 0:
        print(X_test[i])
        print("Actual Label",Y_test[i])
        print("Prediction Lable",y_pred_binary[i])
        print("Prediction",y_pred_onehot[i])
        print("-------------")

were watching how the markets are responding to trumps inauguration
Actual Label 0
Prediction Lable 1
Prediction [0.04681524 0.28519696 0.58792955 0.08005835]
-------------
how theresa mays childhood heroine inspired her taste for leather premium
Actual Label 1
Prediction Lable 0
Prediction [0.12604274 0.705885   0.16027044 0.00780176]
-------------


In [21]:
# Utilize our model and test the real-world headlines
def test(headline):
    headline = cleanText(headline)
    headline_np = np.array([headline])
    indices = sentences_to_indices(headline_np, word_to_index, max_len = maxLen)
    #y_pred_onehot = lstm_model.predict(test_indices)
    y_pred_onehot = BiGRU_model.predict(indices)
    y_pred_binary = onehot_to_binary(y_pred_onehot)
    return True if y_pred_binary == [1] else False

In [22]:
headline = "Here are 10 things you may not know"
test(headline)

True